1. Clip-Higher
Clip-Higher, düşük olasılıklı token'ların olasılığını artırarak entropi çökmesini önlemek için kullanılır. Bu, modelin daha çeşitli cevaplar üretmesini sağlar.

In [29]:
import numpy as np

def soft_clip(probabilities, clip_low=0.2, clip_high=0.3, steepness=20):
    """
    Yumuşak clip fonksiyonu:
    - clip_low altında kalan değerler neredeyse clip_low'ya çekilir.
    - clip_high üzerinde kalan değerler yavaşça bastırılır.
    """
    # Yumuşak minimum
    low_adjusted = clip_low + (probabilities - clip_low) / (1 + np.exp(-steepness*(probabilities - clip_low)))
    # Yumuşak maksimum
    high_adjusted = clip_high + (probabilities - clip_high) / (1 + np.exp(steepness*(probabilities - clip_high)))
    # İki aşamada da uygulayarak dengeli sonuç elde ediyoruz
    clipped = np.where(probabilities < clip_low, low_adjusted, probabilities)
    clipped = np.where(probabilities > clip_high, high_adjusted, clipped)
    return clipped

# Örnek olasılık dağılımı
probabilities = np.array([0.9, 0.05, 0.06])
clipped_probabilities = soft_clip(probabilities)

print("Orijinal Olasılıklar:", probabilities)
print("Soft Clip sonrası Olasılıklar:", clipped_probabilities)

Orijinal Olasılıklar: [0.9  0.05 0.06]
Soft Clip sonrası Olasılıklar: [0.30000369 0.19288612 0.19197462]


2. Dinamik Örnekleme
Dinamik Örnekleme, sıfır gradyanlı örnekleri filtreleyerek eğitim verimliliğini artırır.

In [31]:
# Örnek örnek seti: (olasılık, etiket) şeklinde
samples = [(0.9, 1), (0.1, 0), (0.8, 1), (0.4, 0), (0.55, None)]

def dynamic_sample_filter(samples, lower_bound=0.2, upper_bound=0.8):
    """
    Dinamik örnekleme: 
    Sadece modelin kararsız kaldığı (olasılık değeri belirsiz aralıkta olan) örnekleri seçer.
    """
    # Eğer etiket None ise sadece modelin tahminine göre filtre uyguluyoruz
    filtered = [sample for sample in samples if lower_bound < sample[0] < upper_bound]
    return filtered

filtered_samples = dynamic_sample_filter(samples)
print("Filtrelenmiş Örnekler:", filtered_samples)

Filtrelenmiş Örnekler: [(0.4, 0), (0.55, None)]


3. Token-Level Policy Gradient Loss
Bu teknik, her bir token'ın kaybını ayrı ayrı hesaplayarak uzun cümlelerdeki öğrenmeyi daha etkili hale getirir.

In [33]:
# Örnek token dizisi ve token kayıpları (her token için hata)
tokens = ["token1", "token2", "token3"]
losses = [0.1, 0.4, 0.2]

def token_policy_gradient_loss(tokens, losses, weights=None):
    """
    Token-level kayıp hesaplama:
    - Eğer weights belirtilmemişse tüm tokenlar eşit ağırlıkta kabul edilir.
    - Ağırlıklı ortalama hesaplanır.
    """
    if weights is None:
        weights = [1.0] * len(tokens)
    weighted_loss = sum(l * w for l, w in zip(losses, weights))
    total_weight = sum(weights)
    return weighted_loss / total_weight if total_weight != 0 else 0

# Örneğin, daha kritik tokenlara yüksek ağırlık verilebilir:
weights = [0.8, 1.2, 1.0]
token_level_loss = token_policy_gradient_loss(tokens, losses, weights)
print("Token Düzeyinde Ağırlıklı Kayıp:", token_level_loss)

Token Düzeyinde Ağırlıklı Kayıp: 0.25333333333333335


4. Overlong Reward Shaping
Bu teknik, çok uzun örnekler için ödül gürültüsünü azaltarak eğitim sürecini stabilize eder.

In [35]:
import math

def shaped_reward(length, base_reward=1, max_length=200, steepness=0.1):
    """
    Overlong Reward Shaping:
    - Uzunluk eşik değerini aşarsa, sigmoidal bir ceza uygulanır.
    - Bu, uzunluk arttıkça cezanın yavaşça derinleşmesini sağlar.
    """
    if length <= max_length:
        return base_reward
    # Sigmoid fonksiyon ile ceza: -1 + 2/(1+exp(-steepness*(length-max_length)))
    penalty = -1 + 2 / (1 + math.exp(-steepness * (length - max_length)))
    return base_reward + penalty

# Örnek uzunluk ve ödül
length = 250
reward = shaped_reward(length)
print("Şekillendirilmiş Ödül:", reward)

Şekillendirilmiş Ödül: 1.9866142981514305


In [41]:
!pip install datasets

  Using cached datasets-3.4.1-py3-none-any.whl.metadata (19 kB)
Using cached datasets-3.4.1-py3-none-any.whl (487 kB)


In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from transformers import (
    GPT2LMHeadModel,
    GPT2TokenizerFast,
    DataCollatorForLanguageModeling,
    get_linear_schedule_with_warmup,
)
from torch.optim import AdamW
from datasets import load_dataset

# Model ve Tokenizer
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)

# PAD Token Tanımlama
tokenizer.pad_token = tokenizer.eos_token  # Padding için EOS token'ı kullanıyoruz

# Dataset Yükleme ve Tokenize Etme
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")
dataset = dataset.select(range(2000))  # Küçük bir subset alalım

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, max_length=256, padding="max_length")

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

# Data Collator Kullanarak Padding İşlemi
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
dataloader = DataLoader(tokenized_dataset, batch_size=4, shuffle=True, collate_fn=data_collator)

# Eğitim Parametreleri
epochs = 3
optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = epochs * len(dataloader)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=num_training_steps)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Eğitim Döngüsü
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    for step, batch in enumerate(dataloader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        if step % 50 == 0:
            print(f"Step {step}, Loss: {loss.item():.4f}")

print("Fine-tuning tamamlandı!")

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Epoch 1/3


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step 0, Loss: 8.1641
